In [1]:
!pip install -U spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 21.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
#Para manipular bases de datos
import pandas as pd
#Para limpiar el texto
import re
#Para hacer lemmatizing y stemming
import spacy
#Natural language toolkit, también para procesar lenguaje natural
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
#Para implementar K-means
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
df = pd.read_csv('Datathon 2024 - Reto Hey.csv')

In [5]:
#Para evitar warnings
pd.options.mode.copy_on_write = True

In [6]:
#Para generar un sub-dataframe que sólo contenga los registros con hora
df_con_hora = df[df['time'].notnull()]
df_con_hora.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424 entries, 388 to 811
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    424 non-null    object
 1   time    424 non-null    object
 2   tweet   424 non-null    object
dtypes: object(3)
memory usage: 13.2+ KB


In [7]:
#Para tener una sola columna que contenga la información temporal en el formato correcto
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
#df_con_hora['date'] = pd.to_datetime(df_con_hora['date'])
fecha_base = df_con_hora['date']
df_con_hora['time'] = pd.to_datetime(fecha_base + ' ' + df_con_hora['time'])

In [8]:
#Para eliminar la columna ahora redundante
df_con_hora.drop(df_con_hora.columns[0], axis=1, inplace=True)

In [9]:
#Para procesar el texto, cambiando todo a minúsculas, eliminando caracteres y palabras irrelevantes y tokenizando
stop_words = stopwords.words('spanish')
df_con_hora['tweet'] = df_con_hora['tweet'].apply(lambda x : x.lower())
df_con_hora['tweet'] = df_con_hora['tweet'].apply(lambda x: re.sub('[^A-Za-zÀ-ÿ0-9 \U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639]', ' ', x))
df_con_hora['tweet'] = df_con_hora['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
df_con_hora['tweet'] = df_con_hora['tweet'].apply(word_tokenize)

In [10]:
df_con_hora.head()

,time,tweet
388,2023-06-12 16:52:11,"[gracias, pendiente, información]"
389,2023-06-13 20:36:26,"[feliz, años, santander, free, cambié, heybanc..."
390,2023-06-15 22:51:17,"[hola, caso, comuniqué, 81, 4392, 2626, allí, ..."
391,2023-06-15 18:35:12,"[fácil, now, saque, tarjetas, crédito]"
392,2023-06-16 19:41:04,"[perfecto, iré, punto, hey, gracias, bebé, lol]"


In [11]:
# Función para lemmatizar una lista de tokens en español
def lemmatize_tokens(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

# Función para realizar stemming en una lista de tokens en español
def stem_tokens(tokens):
    return [token.lemma_.lower() for token in nlp(" ".join(tokens))]

In [12]:
# Aplicar la lematización a una columna 'tokens' en un DataFrame
df_con_hora['tweet'] = df_con_hora['tweet'].apply(lemmatize_tokens)

# Aplicar el stemming a una columna 'tokens' en un DataFrame
df_con_hora['tweet'] = df_con_hora['tweet'].apply(stem_tokens)

In [13]:
df_con_hora.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424 entries, 388 to 811
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    424 non-null    datetime64[ns]
 1   tweet   424 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 9.9+ KB


In [14]:
# Crear una matriz TF-IDF de los datos de texto
df_con_hora['tweet'] = df_con_hora['tweet'].apply(lambda x: ' '.join(x))
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_con_hora['tweet'])

# Aplicar K-Means para clusterizar los documentos
k = 5  # Número de clusters
kmeans = KMeans(n_clusters=k)
kmeans.fit(tfidf_matrix)

# Agregar las etiquetas de cluster a los datos originales
df_con_hora['cluster'] = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [15]:
# Imprimir los resultados
df_con_hora['cluster'].value_counts()

cluster
0    231
3     52
2     51
1     51
4     39
Name: count, dtype: int64

In [16]:
df_con_hora.head()

,time,tweet,cluster
388,2023-06-12 16:52:11,gracias pendiente información,3
389,2023-06-13 20:36:26,feliz año santander free cambié heybanco tarje...,0
390,2023-06-15 22:51:17,holar caso comuniqué 81 4392 2626 allí resolve...,0
391,2023-06-15 18:35:12,fácil now sacar tarjeta crédito,0
392,2023-06-16 19:41:04,perfecto ir punto hey gracias bebé lol,4


In [23]:
cluster_0 = df_con_hora[df_con_hora['cluster'] == 0].copy()
cluster_0.head(10)

,time,tweet,cluster
389,2023-06-13 20:36:26,feliz año santander free cambié heybanco tarje...,0
390,2023-06-15 22:51:17,holar caso comuniqué 81 4392 2626 allí resolve...,0
391,2023-06-15 18:35:12,fácil now sacar tarjeta crédito,0
394,2023-06-17 18:00:08,oir 🌝 excelente tiempo respuesta amable person...,0
398,2023-06-19 15:36:18,servie,0
399,2023-06-19 13:32:11,tanto sencillo hacer tipo movimiento celular a...,0
403,2023-06-20 06:47:43,coquetar beneficio real 🔥,0
404,2023-06-20 06:10:33,listo 💪 🏼heybanco palnoe2021,0
409,2023-06-21 16:32:41,si nunca tipo situación método pago hacer incr...,0
411,2023-06-22 15:16:55,buen dia sacar carro usado merkauto autorizar ...,0


In [18]:
cluster_1 = df_con_hora[df_con_hora['cluster'] == 1].copy()
cluster_1.head()

,time,tweet,cluster
423,2023-06-30 19:02:11,abrí contar amar si poder recomeir inversión h...,1
475,2023-07-27 00:01:54,poder empezar tarjeta crédito garantizado pers...,1
488,2023-08-04 23:36:52,fácil abrir cuenta bancario nuevo banco digita...,1
490,2023-08-05 23:09:34,holar buen tarde ayudar confirmar monto máximo...,1
502,2023-08-15 21:50:22,justo pasar mismo momento final volver aparece...,1


In [19]:
cluster_3 = df_con_hora[df_con_hora['cluster'] == 3].copy()
cluster_3.head(10)

,time,tweet,cluster
388,2023-06-12 16:52:11,gracias pendiente información,3
396,2023-06-19 19:52:35,gracias,3
401,2023-06-20 22:22:06,holar mandé dm gracias,3
405,2023-06-21 21:22:49,gracias arreglado,3
406,2023-06-21 16:20:21,gracias 😊 🤗,3
410,2023-06-22 16:25:12,vaaaaa gracias dato,3
416,2023-06-28 17:42:50,gracias,3
417,2023-06-28 04:45:09,gracias hacer quedo 🙌 🏻,3
419,2023-06-28 17:28:12,gracias ahora envío,3
424,2023-07-01 04:07:54,respuesta favorable gracias,3


In [20]:
cluster_4 = df_con_hora[df_con_hora['cluster'] == 4].copy()
cluster_4.head()

,time,tweet,cluster
392,2023-06-16 19:41:04,perfecto ir punto hey gracias bebé lol,4
393,2023-06-16 18:03:36,probado 4 banco tradicional diferente momento ...,4
426,2023-07-03 20:36:54,techreir came súper bien además ahora dar rega...,4
427,2023-07-04 19:32:32,dar gusto pae gran familia finsus saludo 😎 💙,4
436,2023-07-06 02:24:11,😍 nuevo diseño tdc ma mir lón chavo 🙋 🏽 tan pr...,4


In [21]:
cluster_2 = df_con_hora[df_con_hora['cluster'] == 2].copy()
cluster_2.head()

,time,tweet,cluster
395,2023-06-18 17:29:39,listo mucho gracia,2
397,2023-06-19 14:43:55,mucho gracia,2
400,2023-06-20 18:20:40,mucho gracia,2
402,2023-06-20 21:26:18,mucho gracia si entrare levantar repoe,2
407,2023-06-21 22:03:28,mucho gracia resolver gracia sopoe excelente...,2


In [22]:
correlation_mes = df_con_hora['time'].dt.month.corr(df_con_hora['cluster'])
print("La correlación entre el mes y el cluster es de ", correlation_mes)
correlation_dia = df_con_hora['time'].dt.day.corr(df_con_hora['cluster'])
print("La correlación entre el mes y el cluster es de ", correlation_dia)
correlation_hora = df_con_hora['time'].dt.hour.corr(df_con_hora['cluster'])
print("La correlación entre la hora y el cluster es de ", correlation_hora)

La correlación entre el mes y el cluster es de  -0.04427751160799352
La correlación entre el mes y el cluster es de  -0.010451800221875373
La correlación entre la hora y el cluster es de  0.07538188370456936
